In [7]:
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import joblib

In [8]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

# Para cada amostra, identificar o ansatz com maior acurácia
y_best_ansatz = np.argmax(y, axis=1)  # Retorna índices 0-29

In [9]:
# criando kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Definir os parâmetros a testar
param_grid = {
    'n_estimators': [5, 10, 20, 30, 50, 80, 100, 130,150, 180, 200]
}

bagging_clf = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)

grid_search = GridSearchCV(bagging_clf, param_grid, cv=kf, scoring='accuracy', n_jobs=18)

grid_search.fit(X, y_best_ansatz)

# Melhores parâmetros
print("Melhores parâmetros:", grid_search.best_params_)

Melhores parâmetros: {'n_estimators': 100}


In [10]:
# Usar os melhores parâmetros encontrados para validação cruzada
bagging_clf = grid_search.best_estimator_

score = cross_val_score(bagging_clf, X, y_best_ansatz, cv=kf, scoring='accuracy')
print("Scores:", score)
print("Média:", score.mean())
print("Desvio padrão:", score.std())

Scores: [0.3        0.33333333 0.28333333]
Média: 0.3055555555555555
Desvio padrão: 0.02078698548207745


In [11]:
#probabilidades
probabilities = bagging_clf.predict_proba(X[6].reshape(1, -1))  # Shape: (n_amostras, n_classes)

# Top 3 ansatzes mais prováveis para a primeira amostra
top_3 = np.argsort(-probabilities, axis=1)[:, :3]
print(f"Top 3 ansatzes para a amostra: {top_3[0]}")
print(f"Probabilidades: {probabilities[0][top_3[0]]}")

Top 3 ansatzes para a amostra: [ 0  2 14]
Probabilidades: [0.75 0.19 0.02]


In [12]:
# Salvar modelo
joblib.dump(bagging_clf, './models_salvos/Bagging_classifier.joblib')

['./models_salvos/Bagging_classifier.joblib']

Exception ignored in: <function ResourceTracker.__del__ at 0x7e4c05c76de0>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f4e3298ade0>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x79d415986de0>
Traceback (most recent call last):
  File "/usr/lib/python3.13/multiprocessing/reso